# Expedia Hotel Recommendations

# Setup

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 30) # 27 columns of data in training set

# import tensorflow as tf
import matplotlib.pyplot as plt

# from sklearn import datasets, cross_validation, metrics
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
# from sklearn import preprocessing

# Logistic Regression
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# from sklearn.grid_search import GridSearchCV

In [ ]:
def make_features(data):    
    '''
    Extract date-time features from dataframe 'data'.
    Converts date_time, srch_ci, and srch_co fields into
    components (year, month, day, etc) and drops the 
    original field.
    '''
    extract_datetimes(data, 'date_time')
    extract_datetimes(data, 'srch_ci')
    extract_datetimes(data, 'srch_co')
    
    data = data.drop(['date_time', 'srch_ci', 'srch_co'], axis=1)
    
    has_null = ['orig_destination_distance', 'srch_ci_year', 'srch_ci_month', 
                'srch_ci_day', 'srch_ci_hour', 'srch_ci_minute', 
                'srch_ci_dayofyear', 'srch_ci_dayofweek', 'srch_co_year', 
                'srch_co_month', 'srch_co_day', 'srch_co_hour', 'srch_co_minute',
                'srch_co_dayofyear', 'srch_co_dayofweek']
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
    data[has_null] = imp.fit_transform(data[has_null])
    
    return data

def extract_datetimes(data, field):
    data[field] = pd.to_datetime(data[field],errors='coerce')
    data[field+'_year'] = data[field].dt.year
    data[field+'_month'] = data[field].dt.month
    data[field+'_day'] = data[field].dt.day
    data[field+'_hour'] = data[field].dt.hour
    data[field+'_minute'] = data[field].dt.minute
    data[field+'_dayofyear'] = data[field].dt.dayofyear
    data[field+'_dayofweek'] = data[field].dt.dayofweek

def make_PCA(X, n_comp):
    pca = PCA(n_components=n_comp)
    pca.fit(X)
    return pca

In [ ]:
use_PCA = False
save_preds = True

# Make features

In [ ]:
sample_sub = pd.read_csv('data/sample_submission.csv')
train_data = make_features(pd.read_csv('data/train.csv', nrows=10000)) # 37,670,294 total lines
test_data = make_features(pd.read_csv('data/test.csv'))   # 2,528,244 total lines
destinations_data = pd.read_csv('data/destinations.csv', index_col=0)

In [ ]:
# train_data #['srch_destination_id'].apply()

In [ ]:
# destinations_data

In [ ]:
# destinations_data[destinations_data['srch_destination_id']==8250]

In [ ]:
# d = {'user_id':1, 'user':'user1', 'group_id':3, 'group_name':'ordinary users'}
# m = {'user_id':'uid', 'group_id':'gid', 'group_name':'group'}

In [ ]:
# b = dict((m.get(k, k), v) for (k, v) in d.items())

In [ ]:
# m.get?

In [ ]:
# train_data['srch_destination_id'].get(0)

In [ ]:
# destinations_data.ix[8250]

In [ ]:
# train_data['srch_destination_id'][0]

In [ ]:
# destinations_data[destinations_data['srch_destination_id']==train_data['srch_destination_id'][3]]

In [ ]:
features = ['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id',
       'hotel_continent', 'hotel_country', 'hotel_market',
       'date_time_year', 'date_time_month', 'date_time_day', 'date_time_hour',
       'date_time_minute', 'date_time_dayofyear', 'date_time_dayofweek',
       'srch_ci_year', 'srch_ci_month', 'srch_ci_day', 'srch_ci_hour',
       'srch_ci_minute', 'srch_ci_dayofyear', 'srch_ci_dayofweek',
       'srch_co_year', 'srch_co_month', 'srch_co_day', 'srch_co_hour',
       'srch_co_minute', 'srch_co_dayofyear', 'srch_co_dayofweek']

# 'srch_ci', 'srch_co', 'orig_destination_distance', 'is_booking', 'cnt',

In [ ]:
X_all = train_data.ix[:,features]
y_all = train_data.ix[:,'hotel_cluster']
X_test = test_data.ix[:,features]

In [ ]:
# # checking correlation of features
# plt.matshow(X_all.corr())
# plt.colorbar()

In [ ]:
if use_PCA:
    X = X_all.as_matrix()
    pca = make_PCA(X, 15)
    X = pca.transform(X)
else:
    X = X_all    

In [ ]:
# generate training and cross-validation features
X_train, X_cv, y_train, y_cv = train_test_split(X, y_all, train_size=.7, random_state=1)

# Train: Logistic Regression Model

In [ ]:
expedia_lr = LogisticRegression(multi_class='ovr', 
                                solver='newton-cg', 
                                max_iter=1000, 
                                penalty='l2')

In [ ]:
%time expedia_lr = expedia_lr.fit(X_train, y_train)

In [ ]:
%%time

score_train = expedia_lr.score(X_train, y_train)
score_cv = expedia_lr.score(X_cv, y_cv)

# train/cv
print ('Training Score:', score_train, ', CV Score:', score_cv) 

# Predict

In [ ]:
if use_PCA:
    X_test = pca.transform(X_test.as_matrix())

In [ ]:
# break the test set into n = num_split sets to predict on
# split_size = 0
num_splits = 10
n_test = X_test.shape[0]
top_pred_hotel_cluster = []

In [ ]:
%%time

for i in range(num_splits):
    prob_prediction = expedia_lr.predict_proba(X_test[int(i*n_test/num_splits):int((i+1)*n_test/num_splits)])
    top_pred_hotel_cluster.append([' '.join([str(hotel) for hotel in row]) for row in np.argsort(prob_prediction)[:,-5:]])

In [ ]:
# %%time 
# prob_prediction = expedia_lr.predict_proba(X_test[:split_size])
# top_pred_hotel_cluster.append([' '.join([str(hotel) for hotel in row]) for row in np.argsort(prob_prediction)[:,-5:]])

In [ ]:
a = np.array(top_pred_hotel_cluster)

In [ ]:
submission = pd.DataFrame(np.concatenate((a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9]), axis=0), columns=['hotel_cluster'])

In [ ]:
submission.to_csv('expedia-rf-2016-04-30-s1.csv', index_label='Id')

# Plot learning curves

In [ ]:
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.datasets import load_digits
from sklearn import cross_validation
from sklearn.learning_curve import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and traning learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : integer, cross-validation generator, optional
        If an integer is passed, it is the number of folds (defaults to 3).
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [ ]:
%%time 

estimator = expedia_lr

# from sklearn.ensemble import GradientBoostingClassifier
# estimator = GradientBoostingClassifier()
title = "Learning Curves (Logistic Regression)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = cross_validation.ShuffleSplit(X.shape[0], n_iter=5,
                                   test_size=0.3, random_state=0)

plot_learning_curve(estimator, title, X, y_all, n_jobs=2, cv=None)
plt.show()

# Exhaustive Grid Search

In [ ]:
# Split again, generate training and cross-validation features for grid search
X_grid_train, X_grid_cv, y_grid_train, y_grid_cv = cross_validation.train_test_split(X_train, 
                                                                                     y_train, 
                                                                                     test_size=0.40, 
                                                                                     random_state=1)

In [ ]:
param_grid = [
    {'n_estimators': [200], 'min_samples_split': [1, 2]}
]
scores = ['precision', 'recall']
# , 'max_features': [2, 3, 5]

In [ ]:
%%time
clf = GridSearchCV(RandomForestClassifier(), param_grid, error_score=0, n_jobs=1)
clf.fit(X_grid_train, y_grid_train)

print(clf.best_score_, clf.best_params_)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))

In [ ]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(OneVsRestClassifier(SVC()), param_grid,
                       scoring='%s_weighted' % score)
    clf.fit(X_grid_train, y_grid_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_grid_cv, clf.predict(X_grid_cv)
    print(classification_report(y_true, y_pred))
    print()